In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry
import datetime
import time
from collections import Counter
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from intervals import Interval
import warnings
warnings.filterwarnings('ignore')

## 垃圾数据处理

In [3]:
# 数据预处理
data=pd.read_csv('../data/Hoorn/Hoorn_OpenLitterMap.csv')

In [4]:
df = data.fillna(0)# 填充缺失值

In [5]:
df

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,...,ear_plugs,elec_small,elec_large,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3
0,118956,2,iPhone 6,2021-02-03 14:05:54,52.651031,5.081417,Yes,"1, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
1,118957,2,iPhone 6,2021-02-03 13:54:23,52.649972,5.082583,Yes,"18, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
2,118958,2,iPhone 6,2021-02-03 14:06:30,52.650917,5.081553,Yes,"2, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
3,118959,2,iPhone 6,2021-02-03 14:02:00,52.651183,5.082656,Yes,"47, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
4,118960,2,iPhone 6,2021-02-03 14:01:17,52.651181,5.082553,Yes,"29, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19622,279325,2,iPhone 13 mini,2022-03-20 09:46:20,52.652412,5.081511,Yes,"43, Ekster, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
19623,279326,2,iPhone 13 mini,2022-03-20 09:45:09,52.652428,5.080361,Yes,"74, Lijster, Kersenboogerd, Hoorn, North Holla...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
19624,281636,2,iPhone 13 mini,2022-03-20 10:00:23,52.654053,5.079644,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,brand:oasis,0,0.0
19625,281638,2,iPhone 13 mini,2022-03-20 09:59:52,52.654020,5.079669,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0


### 垃圾分类

In [6]:
# 垃圾分类
def sumc(x):# 定义合并类的函数
    count_sum = 0
    for k in x.iteritems():
        count_sum += k[1]
    return count_sum

In [7]:
df['SMOKING']=df.loc[:,'butts':'smokingOther'].apply(sumc,axis=1)
df['SMOKING'].sum()

1074.0

In [8]:
df=df.drop(df.loc[:,'butts':'smokingOther'],axis=1)

In [9]:
df['FOOD']=df.loc[:,'sweetWrappers':'chewing_gum'].apply(sumc,axis=1)
df['FOOD'].sum()

2579.0

In [10]:
df=df.drop(df.loc[:,'sweetWrappers':'chewing_gum'],axis=1)

In [11]:
df['COFFEE']=df.loc[:,'coffeeCups':'coffeeOther'].apply(sumc,axis=1)
df['COFFEE'].sum()

559.0

In [12]:
df=df.drop(df.loc[:,'coffeeCups':'coffeeOther'],axis=1)

In [13]:
df['ALCOHOL']=df.loc[:,'beerCan':'alcoholOther'].apply(sumc,axis=1)
df['ALCOHOL'].sum()

1276.0

In [14]:
df=df.drop(df.loc[:,'beerCan':'alcoholOther'],axis=1)

In [15]:
df['SOFTDRINKS']=df.loc[:,'waterBottle':'softDrinkOther'].apply(sumc,axis=1)
df['SOFTDRINKS'].sum()

6208.0

In [16]:
df=df.drop(df.loc[:,'waterBottle':'softDrinkOther'],axis=1)

In [17]:
df['SANITARY']=df.loc[:,'condoms':'hand_sanitiser'].apply(sumc,axis=1)
df['SANITARY'].sum()

997.0

In [18]:
df=df.drop(df.loc[:,'condoms':'hand_sanitiser'],axis=1)

In [19]:
df['COASTAL']=df.loc[:,'microplastics':'styro_large'].apply(sumc,axis=1)
df['COASTAL'].sum()

327.0

In [20]:
df=df.drop(df.loc[:,'microplastics':'styro_large'],axis=1)

In [21]:
df['DUMPING']=df.loc[:,'small':'large'].apply(sumc,axis=1)
df['DUMPING'].sum()

0.0

In [22]:
df=df.drop(df.loc[:,'small':'large'],axis=1)

In [23]:
df['INDUSTRIAL']=df.loc[:,'oil':'industrial_other'].apply(sumc,axis=1)
df['INDUSTRIAL'].sum()

0.0

In [24]:
df['BRANDS']=df.loc[:,'aadrink':'wrigleys'].apply(sumc,axis=1)
df['BRANDS'].sum()

4751.0

In [25]:
df=df.drop(df.loc[:,'aadrink':'wrigleys'],axis=1)

In [26]:
df=df.drop(df.loc[:,'oil':'industrial_other'],axis=1)

In [27]:
df['DOGSHIT']=df.loc[:,'poo':'poo_in_bag'].apply(sumc,axis=1)
df['DOGSHIT'].sum()

0.0

In [28]:
df=df.drop(df.loc[:,'poo':'poo_in_bag'],axis=1)

In [29]:
df['ART']=df.loc[:,'item']

In [30]:
df['ART'].sum()

0.0

In [31]:
df=df.drop('item',axis=1)

In [32]:
df['OTHER']=df.loc[:,'random_litter':'other'].apply(sumc,axis=1)
df['OTHER'].sum()

7309.0

In [33]:
df=df.drop(df.loc[:,'random_litter':'other'],axis=1)

In [34]:
df.columns

Index(['id', 'verification', 'phone', 'datetime', 'lat', 'lon', 'picked up',
       'address', 'total_litter', 'SMOKING', 'FOOD', 'COFFEE', 'ALCOHOL',
       'SOFTDRINKS', 'SANITARY', 'COASTAL', 'DUMPING', 'INDUSTRIAL', 'BRANDS',
       'DOGSHIT', 'ART', 'OTHER', 'custom_tag_1', 'custom_tag_2',
       'custom_tag_3'],
      dtype='object')

In [35]:
df=df.drop(df.loc[:,'custom_tag_1':'custom_tag_3'],axis=1)# 删除不需要的标签

In [36]:
df['OTHER']=df['OTHER']+df['ART']+df['DOGSHIT']+df['INDUSTRIAL']+df['DUMPING']

In [37]:
df['OTHER'].sum()

7309.0

In [38]:
df=df.drop(['ART','DOGSHIT','INDUSTRIAL','DUMPING'],axis=1)

In [39]:
df # 分类完毕

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
0,118956,2,iPhone 6,2021-02-03 14:05:54,52.651031,5.081417,Yes,"1, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,118957,2,iPhone 6,2021-02-03 13:54:23,52.649972,5.082583,Yes,"18, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118958,2,iPhone 6,2021-02-03 14:06:30,52.650917,5.081553,Yes,"2, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,118959,2,iPhone 6,2021-02-03 14:02:00,52.651183,5.082656,Yes,"47, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,118960,2,iPhone 6,2021-02-03 14:01:17,52.651181,5.082553,Yes,"29, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19622,279325,2,iPhone 13 mini,2022-03-20 09:46:20,52.652412,5.081511,Yes,"43, Ekster, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
19623,279326,2,iPhone 13 mini,2022-03-20 09:45:09,52.652428,5.080361,Yes,"74, Lijster, Kersenboogerd, Hoorn, North Holla...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19624,281636,2,iPhone 13 mini,2022-03-20 10:00:23,52.654053,5.079644,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19625,281638,2,iPhone 13 mini,2022-03-20 09:59:52,52.654020,5.079669,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [40]:
df=df[-df['datetime'].str.contains('2020')].reset_index(drop=True) # 删除2020年的零散数据

In [41]:
df

,id,verification,phone,datetime,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
0,118956,2,iPhone 6,2021-02-03 14:05:54,52.651031,5.081417,Yes,"1, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,118957,2,iPhone 6,2021-02-03 13:54:23,52.649972,5.082583,Yes,"18, Amstelweg, Kersenboogerd, Hoorn, North Hol...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118958,2,iPhone 6,2021-02-03 14:06:30,52.650917,5.081553,Yes,"2, Stijl, Kersenboogerd, Hoorn, North Holland,...",2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,118959,2,iPhone 6,2021-02-03 14:02:00,52.651183,5.082656,Yes,"47, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,118960,2,iPhone 6,2021-02-03 14:01:17,52.651181,5.082553,Yes,"29, Stijl, Kersenboogerd, Hoorn, North Holland...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,279325,2,iPhone 13 mini,2022-03-20 09:46:20,52.652412,5.081511,Yes,"43, Ekster, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
19608,279326,2,iPhone 13 mini,2022-03-20 09:45:09,52.652428,5.080361,Yes,"74, Lijster, Kersenboogerd, Hoorn, North Holla...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19609,281636,2,iPhone 13 mini,2022-03-20 10:00:23,52.654053,5.079644,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19610,281638,2,iPhone 13 mini,2022-03-20 09:59:52,52.654020,5.079669,Yes,"Spaarneweg, Kersenboogerd, Hoorn, North Hollan...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# df.to_csv('./outdata/Hoorn_litter.csv')

### 合并时间

In [46]:
df['datetime']=pd.to_datetime(df['datetime'])

In [47]:
Date=df['datetime'].dt.date
df_dateconcat=df.loc[:,'total_litter':'OTHER'].groupby(Date).sum() # 合并同日期的垃圾数量
df_dateconcat

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
datetime,,,,,,,,,,
2021-01-06,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2021-01-07,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2021-01-08,114.0,2.0,13.0,6.0,3.0,34.0,9.0,0.0,25.0,32.0
2021-01-09,58.0,3.0,11.0,1.0,0.0,12.0,3.0,0.0,11.0,21.0
2021-01-10,49.0,2.0,2.0,0.0,8.0,20.0,0.0,0.0,18.0,9.0
...,...,...,...,...,...,...,...,...,...,...
2021-12-19,92.0,19.0,10.0,1.0,5.0,28.0,7.0,0.0,28.0,22.0
2021-12-20,232.0,5.0,22.0,7.0,7.0,60.0,17.0,0.0,50.0,114.0
2021-12-21,14.0,1.0,4.0,0.0,0.0,7.0,0.0,0.0,5.0,2.0


In [48]:
df_dateconcat.max()

total_litter    350.0
SMOKING          86.0
FOOD             43.0
COFFEE           15.0
ALCOHOL          34.0
SOFTDRINKS       82.0
SANITARY         25.0
COASTAL          38.0
BRANDS           68.0
OTHER           190.0
dtype: float64

### 对缺失的日期补0

In [49]:
start = datetime.datetime(2021, 1, 1)
end = datetime.datetime(2022, 4, 1)
date_range = pd.date_range(start, end)

In [50]:
date_range

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10',
               ...
               '2022-03-23', '2022-03-24', '2022-03-25', '2022-03-26',
               '2022-03-27', '2022-03-28', '2022-03-29', '2022-03-30',
               '2022-03-31', '2022-04-01'],
              dtype='datetime64[ns]', length=456, freq='D')

In [51]:
# 筛选缺失的日期
df_date=pd.to_datetime(df_dateconcat.index) 
dateflag=date_range.isin(df_date) 

In [52]:
# date_all=np.concatenate((pd.Series(df_dateconcat.index).values,pd.Series(date_range[~dateflag].date).values),axis=0)# 合并两个时间
# date_all.size
# date_all与date_range相同，弃用data_all

In [53]:
np.zeros(228*9).reshape(228,9).shape

(228, 9)

In [54]:
# 缺失的数据
dfdif=pd.DataFrame(np.zeros(228*9).reshape(228,9),
                   index=[pd.Series(date_range[~dateflag].date).values],
                   columns=['total_litter','SMOKING','FOOD','COFFEE','ALCOHOL','SOFTDRINKS','SANITARY','COASTAL','OTHER'])

In [55]:
df_dateall=pd.concat([df_dateconcat,dfdif])# 合并

In [56]:
df_dateall=df_dateall.loc[:,'total_litter':'OTHER'].groupby(date_range).sum()

In [57]:
df_dateall

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
2021-01-01,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2021-01-02,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2021-01-03,114.0,2.0,13.0,6.0,3.0,34.0,9.0,0.0,25.0,32.0
2021-01-04,58.0,3.0,11.0,1.0,0.0,12.0,3.0,0.0,11.0,21.0
2021-01-05,49.0,2.0,2.0,0.0,8.0,20.0,0.0,0.0,18.0,9.0
...,...,...,...,...,...,...,...,...,...,...
2022-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
df_dateall.max()

total_litter    350.0
SMOKING          86.0
FOOD             43.0
COFFEE           15.0
ALCOHOL          34.0
SOFTDRINKS       82.0
SANITARY         25.0
COASTAL          38.0
BRANDS           68.0
OTHER           190.0
dtype: float64

### 将垃圾数据按天划分

In [59]:
df.datetime

0       2021-02-03 14:05:54
1       2021-02-03 13:54:23
2       2021-02-03 14:06:30
3       2021-02-03 14:02:00
4       2021-02-03 14:01:17
                ...        
19607   2022-03-20 09:46:20
19608   2022-03-20 09:45:09
19609   2022-03-20 10:00:23
19610   2022-03-20 09:59:52
19611   2022-03-20 09:47:22
Name: datetime, Length: 19612, dtype: datetime64[ns]

In [60]:
df=df.set_index('datetime').sort_index()

In [61]:
df.to_csv('./outdata/Hoorn_litter_sorted')

In [62]:
result = [group[1] for group in df.groupby(df.index.date)]# 将结果按天分组并只打印键值对的值

In [63]:
result[1]

,id,verification,phone,lat,lon,picked up,address,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER
datetime,,,,,,,,,,,,,,,,,
2021-01-07 09:24:45,131285,2,iPhone 6,52.653561,5.085186,Yes,"22, Bolwerk, Kersenboogerd, Hoorn, North Holla...",2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## 天气时间数据处理

In [64]:
weather = pd.read_csv('../data/Hoorn/Hoorn_Weather.csv')

In [65]:
weather['date']=pd.to_datetime(weather['date'])

In [66]:
weather['weather'].value_counts()

Sunny      212
Cloudy     112
Showers     98
Foggy       30
Snow         4
Name: weather, dtype: int64

In [67]:
wtcdt=pd.read_csv('../data/Hoorn/EHAM.csv')

In [68]:
wtcdt['date']=pd.to_datetime(wtcdt['date'])

In [69]:
wtcdt = pd.DataFrame(wtcdt).set_index('date') # 将时间作为索引

In [70]:
wtcdt

,Actual Mean Temperature,Actual Min Temperature,Actual Max Temperature,Average Min Temperature,Average Max Temperature,Actual Max Wind Speed,Actual Dew Point,Actual High Dew Point,Actual Low Dew Point,Actual Sea Level Pressure
date,,,,,,,,,,
2021-01-01,35.67,30,43,30,40,8,34.09,39,28,29.84
2021-01-02,36.71,34,39,31,40,8,36.27,39,32,29.96
2021-01-03,38.61,37,39,31,40,16,34.67,37,32,30.08
2021-01-04,38.00,37,39,31,40,22,33.87,36,30,30.11
2021-01-05,37.33,36,39,31,40,16,34.52,36,32,30.11
...,...,...,...,...,...,...,...,...,...,...
2022-03-28,47.94,41,59,35,51,9,42.79,45,41,30.37
2022-03-29,47.42,43,55,35,51,13,40.79,43,39,30.08
2022-03-30,43.79,37,48,36,51,16,36.26,39,32,29.81


### 将天气状况加入补全后数据

In [71]:
M_test=pd.concat([df_dateall,wtcdt],axis=1)# 横向合并

In [72]:
M_test

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER,Actual Mean Temperature,Actual Min Temperature,Actual Max Temperature,Average Min Temperature,Average Max Temperature,Actual Max Wind Speed,Actual Dew Point,Actual High Dew Point,Actual Low Dew Point,Actual Sea Level Pressure
2021-01-01,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,35.67,30,43,30,40,8,34.09,39,28,29.84
2021-01-02,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,36.71,34,39,31,40,8,36.27,39,32,29.96
2021-01-03,114.0,2.0,13.0,6.0,3.0,34.0,9.0,0.0,25.0,32.0,38.61,37,39,31,40,16,34.67,37,32,30.08
2021-01-04,58.0,3.0,11.0,1.0,0.0,12.0,3.0,0.0,11.0,21.0,38.00,37,39,31,40,22,33.87,36,30,30.11
2021-01-05,49.0,2.0,2.0,0.0,8.0,20.0,0.0,0.0,18.0,9.0,37.33,36,39,31,40,16,34.52,36,32,30.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.94,41,59,35,51,9,42.79,45,41,30.37
2022-03-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.42,43,55,35,51,13,40.79,43,39,30.08
2022-03-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.79,37,48,36,51,16,36.26,39,32,29.81
2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.29,32,39,36,51,26,34.13,37,32,29.66


### 将天气状况加入未补全数据

In [73]:
# 匹配时间
df_dateconcat.index

Index([2021-01-06, 2021-01-07, 2021-01-08, 2021-01-09, 2021-01-10, 2021-01-11,
       2021-01-12, 2021-01-13, 2021-01-14, 2021-01-31,
       ...
       2021-12-13, 2021-12-15, 2021-12-16, 2021-12-17, 2021-12-18, 2021-12-19,
       2021-12-20, 2021-12-21, 2022-01-20, 2022-03-20],
      dtype='object', name='datetime', length=228)

In [74]:
isindex1=wtcdt.index.isin(df_dateconcat.index)

In [75]:
wtcdt[isindex1]

,Actual Mean Temperature,Actual Min Temperature,Actual Max Temperature,Average Min Temperature,Average Max Temperature,Actual Max Wind Speed,Actual Dew Point,Actual High Dew Point,Actual Low Dew Point,Actual Sea Level Pressure
date,,,,,,,,,,
2021-01-06,36.59,34,39,31,40,15,32.61,36,32,30.08
2021-01-07,37.54,34,41,31,40,16,35.33,37,32,29.96
2021-01-08,35.96,30,41,31,40,7,34.09,36,30,30.13
2021-01-09,32.43,25,43,31,40,8,29.57,37,25,30.28
2021-01-10,36.24,28,41,31,40,12,33.07,37,27,30.28
...,...,...,...,...,...,...,...,...,...,...
2021-12-19,45.07,43,46,33,42,12,43.57,45,41,30.64
2021-12-20,41.63,32,45,33,42,7,35.17,39,30,30.40
2021-12-21,31.80,27,43,33,42,5,28.54,32,27,30.37


In [76]:
isindex2=weather['date'].isin(df_dateconcat.index)

In [77]:
isindex2.value_counts()

False    228
True     228
Name: date, dtype: int64

In [78]:
wtdf = pd.DataFrame(weather[isindex2]).set_index('date')

In [79]:
M=pd.concat([df_dateconcat,wtdf,wtcdt[isindex1]],axis=1)# 横向合并

In [80]:
M

,total_litter,SMOKING,FOOD,COFFEE,ALCOHOL,SOFTDRINKS,SANITARY,COASTAL,BRANDS,OTHER,...,Actual Mean Temperature,Actual Min Temperature,Actual Max Temperature,Average Min Temperature,Average Max Temperature,Actual Max Wind Speed,Actual Dew Point,Actual High Dew Point,Actual Low Dew Point,Actual Sea Level Pressure
2021-01-06,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,36.59,34,39,31,40,15,32.61,36,32,30.08
2021-01-07,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,37.54,34,41,31,40,16,35.33,37,32,29.96
2021-01-08,114.0,2.0,13.0,6.0,3.0,34.0,9.0,0.0,25.0,32.0,...,35.96,30,41,31,40,7,34.09,36,30,30.13
2021-01-09,58.0,3.0,11.0,1.0,0.0,12.0,3.0,0.0,11.0,21.0,...,32.43,25,43,31,40,8,29.57,37,25,30.28
2021-01-10,49.0,2.0,2.0,0.0,8.0,20.0,0.0,0.0,18.0,9.0,...,36.24,28,41,31,40,12,33.07,37,27,30.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,92.0,19.0,10.0,1.0,5.0,28.0,7.0,0.0,28.0,22.0,...,45.07,43,46,33,42,12,43.57,45,41,30.64
2021-12-20,232.0,5.0,22.0,7.0,7.0,60.0,17.0,0.0,50.0,114.0,...,41.63,32,45,33,42,7,35.17,39,30,30.40
2021-12-21,14.0,1.0,4.0,0.0,0.0,7.0,0.0,0.0,5.0,2.0,...,31.80,27,43,33,42,5,28.54,32,27,30.37
2022-01-20,5.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,...,39.59,36,43,31,40,30,32.93,37,28,30.52


In [81]:
M_train=M.drop(M.loc[:,'SMOKING':'OTHER'],axis=1)# 舍弃分类

In [82]:
M_train

,total_litter,weather,Actual Mean Temperature,Actual Min Temperature,Actual Max Temperature,Average Min Temperature,Average Max Temperature,Actual Max Wind Speed,Actual Dew Point,Actual High Dew Point,Actual Low Dew Point,Actual Sea Level Pressure
2021-01-06,3.0,Sunny,36.59,34,39,31,40,15,32.61,36,32,30.08
2021-01-07,2.0,Showers,37.54,34,41,31,40,16,35.33,37,32,29.96
2021-01-08,114.0,Showers,35.96,30,41,31,40,7,34.09,36,30,30.13
2021-01-09,58.0,Foggy,32.43,25,43,31,40,8,29.57,37,25,30.28
2021-01-10,49.0,Cloudy,36.24,28,41,31,40,12,33.07,37,27,30.28
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,92.0,Showers,45.07,43,46,33,42,12,43.57,45,41,30.64
2021-12-20,232.0,Cloudy,41.63,32,45,33,42,7,35.17,39,30,30.40
2021-12-21,14.0,Sunny,31.80,27,43,33,42,5,28.54,32,27,30.37
2022-01-20,5.0,Cloudy,39.59,36,43,31,40,30,32.93,37,28,30.52


### 对天气描述编码

In [83]:
wtdf

,weather
date,
2021-01-06,Sunny
2021-01-07,Showers
2021-01-08,Showers
2021-01-09,Foggy
2021-01-10,Cloudy
...,...
2021-12-19,Showers
2021-12-20,Cloudy
2021-12-21,Sunny


In [84]:
values_wt=wtdf['weather']

In [85]:
# 整数编码
label_encoder_wt = LabelEncoder()
integer_encoded_wt = label_encoder_wt.fit_transform(values_wt)
integer_encoded_wt

array([3, 2, 2, 1, 0, 0, 2, 3, 3, 3, 2, 2, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 0, 3, 0, 2, 3, 0, 2, 0, 0, 0, 0, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0, 0,
       3, 2, 3, 0, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3,
       3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 3, 2, 0,
       3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3,
       3, 3, 0, 2, 0, 3, 3, 1, 0, 2, 3, 2, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 2, 0, 2, 2, 2, 3, 3, 0, 1, 3,
       1, 3, 2, 3, 0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 3, 3, 2, 1, 1, 0, 0, 0,
       0, 1, 2, 0, 1, 3, 0, 2, 3, 0, 2, 2, 2, 1, 1, 2, 3, 2, 3, 2, 0, 0,
       1, 2, 1, 2, 0, 3, 0, 3])

In [86]:
# 二进制编码
wt_encoded = OneHotEncoder(sparse=False)
integer_encoded_wt = integer_encoded_wt.reshape(len(integer_encoded_wt), 1)
wt_encoded = wt_encoded.fit_transform(integer_encoded_wt)
wt_encoded.shape

(228, 4)

In [87]:
# 第一个例子的反转
inverted_wt = label_encoder_wt.inverse_transform([argmax(wt_encoded[0, :])])
inverted_wt

array(['Sunny'], dtype=object)

## poi处理

In [88]:
points=gpd.read_file('../data/Hoorn/shape_large/points.shp')

In [89]:
public=points['type'].isin(['post_box', 'pub',  'school', 'social_facility','charging_station','childcare','arts_centre',

'community_centre','kindergarten','library',  'elevator','fire_station','post_depot', 

 'social_centre', 'marketplace','police', 'theatre','yard','music_school','townhall'])

In [90]:
poi_public=points['type'][public].value_counts()

In [91]:
poi_public=poi_public.values.sum()

In [92]:
poi_public

127

In [93]:
points['type'][public]='public'

In [94]:
health=points['type'].isin(['dentist','doctors','pharmacy', 'veterinary','hospital', 'clinic'])

In [95]:
poi_health=points['type'][health].value_counts()

In [96]:
poi_health=poi_health.values.sum()

In [97]:
points['type'][health]='health'

In [98]:
leisure=points['type'].isin(['cafe', 'cinema','casino'])

In [99]:
poi_leisure=points['type'][leisure].value_counts()

In [100]:
poi_leisure=poi_leisure.values.sum()

In [101]:
poi_leisure

14

In [102]:
points['type'][leisure]='leisure'

In [103]:
catering=points['type'].isin([ 'restaurant','fast_food'])

In [104]:
poi_catering=points['type'][catering].value_counts()

In [105]:
poi_catering=poi_catering.values.sum()

In [106]:
poi_catering

107

In [107]:
points['type'][catering]='catering'

In [108]:
accommodation=points['type'].isin(['hotel','shelter','caravan_site'])

In [109]:
poi_accommodation=points['type'][accommodation].value_counts()

In [110]:
poi_accommodation=poi_accommodation.values.sum()

In [111]:
poi_accommodation

15

In [112]:
points['type'][accommodation]='accommodation'

In [113]:
shopping=points['type'].isin(['car_wash','vending_machine', 'car_sharing','ice_cream','parking', 'shower','bicycle_repair_s', 'car_rental','taxi'])

In [114]:
poi_shopping=points['type'][shopping].value_counts()

In [115]:
poi_shopping=poi_shopping.values.sum()

In [116]:
poi_shopping

29

In [117]:
points['type'][shopping]='shopping'

In [118]:
money=points['type'].isin(['atm','bank'])

In [119]:
poi_money=points['type'][money].value_counts()

In [120]:
poi_money=poi_money.values.sum()

In [121]:
poi_money

15

In [122]:
points['type'][money]='money'

In [123]:
tourism=points['type'].isin(['artwork', 'memorial', 'gallery','museum','attraction', 'cannon',  'archaeological_s',  'viewpoint', 'picnic_site'])

In [124]:
poi_tourism=points['type'][tourism].value_counts()

In [125]:
poi_tourism=poi_tourism.values.sum()

In [126]:
poi_tourism

140

In [127]:
points['type'][tourism]='tourism'

In [128]:
miscpoi=points['type'].isin(['bench','waste_basket','tower','toilets','surveillance','works','drinking_water', 'windmill','fountain', 'water_well'])

In [129]:
poi_miscpoi=points['type'][miscpoi].value_counts()

In [130]:
poi_miscpoi=poi_miscpoi.values.sum()

In [131]:
poi_miscpoi

1131

In [132]:
points['type'][miscpoi]='miscpoi'

In [133]:
other=points['type'].isin(['construction','give_way','clock','street_cabinet', 'studio', 'place_of_worship',  'crane',  'communications_t','flagpole', 'chimney', 'lighthouse','tram_stop', 'yes'])

In [134]:
poi_other=points['type'][other].value_counts()

In [135]:
poi_other=poi_other.values.sum()

In [136]:
poi_other

19

In [137]:
points['type'][other]='other'

In [138]:
traffic=points['type'].isin(['mast','bicycle_parking','crossing', 'recycling', 'traffic_signals', 'signal','switch', 'level_crossing','motorway_junctio','ferry_terminal','turning_circle', 'buffer_stop', 'street_lamp','fuel', 'parking_entrance', 'stop', 'station','milestone','mini_roundabout',  'railway_crossing','junction'])

In [139]:
poi_traffic=points['type'][traffic].value_counts()

In [140]:
poi_traffic=poi_traffic.values.sum()

In [141]:
poi_traffic

620

In [142]:
points['type'][traffic]='traffic'

In [146]:
len(points['type'].value_counts())

13

In [148]:
points[:-2]

,osm_id,timestamp,name,type,geometry
0,34050780,None,Museum van de Twintigste Eeuw,tourism,POINT (5.06242 52.63508)
1,47051205,None,Avenhorn,traffic,POINT (5.00851 52.63502)
2,47056631,None,None,traffic,POINT (5.05786 52.63695)
3,47060139,None,None,traffic,POINT (5.05724 52.63817)
4,47069901,None,None,traffic,POINT (5.05239 52.64179)
...,...,...,...,...,...
2395,804881163,None,None,traffic,POINT (5.04714 52.64798)
2396,804881164,None,None,traffic,POINT (5.04729 52.64775)
2397,804881165,None,None,traffic,POINT (5.04726 52.64767)
2398,804881166,None,None,traffic,POINT (5.04692 52.64760)


In [149]:
points[:-2].to_file('./outdata/poi_all/points.shp')

In [145]:
poi=pd.DataFrame(np.array([poi_public,poi_health,poi_leisure,poi_catering,poi_accommodation,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other]).reshape(1,11),
                 columns=['poi_public','poi_health','poi_leisure','poi_catering','poi_accommodation','poi_shopping','poi_money','poi_tourism','poi_miscpoi','poi_traffic','poi_other'],
                 index=['counter']
                )

In [146]:
poi

,poi_public,poi_health,poi_leisure,poi_catering,poi_accommodation,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
counter,127,37,14,107,15,29,15,140,1131,620,19


In [147]:
# poi_other=poi_other+poi_money+poi_shopping+poi_accommodation+poi_health+poi_leisure

In [148]:
# poi['poi_other']=poi_other

In [149]:
# poi=poi.drop(['poi_health','poi_leisure','poi_accommodation','poi_shopping','poi_money'],axis=1)

In [150]:
# poi

In [151]:
poi_array=np.array(poi)

In [152]:
poi_array=np.tile(poi,(228,1))

In [153]:
poi_array

array([[ 127,   37,   14, ..., 1131,  620,   19],
       [ 127,   37,   14, ..., 1131,  620,   19],
       [ 127,   37,   14, ..., 1131,  620,   19],
       ...,
       [ 127,   37,   14, ..., 1131,  620,   19],
       [ 127,   37,   14, ..., 1131,  620,   19],
       [ 127,   37,   14, ..., 1131,  620,   19]])

In [154]:
poi_df=pd.DataFrame(poi_array,columns=poi.columns,index=M_train.index)

In [155]:
poi_df

,poi_public,poi_health,poi_leisure,poi_catering,poi_accommodation,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2021-01-06,127,37,14,107,15,29,15,140,1131,620,19
2021-01-07,127,37,14,107,15,29,15,140,1131,620,19
2021-01-08,127,37,14,107,15,29,15,140,1131,620,19
2021-01-09,127,37,14,107,15,29,15,140,1131,620,19
2021-01-10,127,37,14,107,15,29,15,140,1131,620,19
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,127,37,14,107,15,29,15,140,1131,620,19
2021-12-20,127,37,14,107,15,29,15,140,1131,620,19
2021-12-21,127,37,14,107,15,29,15,140,1131,620,19
2022-01-20,127,37,14,107,15,29,15,140,1131,620,19


In [156]:
M_train=pd.concat([M_train,poi_df],axis=1)

In [157]:
M_train

,total_litter,weather,Actual Mean Temperature,Actual Min Temperature,Actual Max Temperature,Average Min Temperature,Average Max Temperature,Actual Max Wind Speed,Actual Dew Point,Actual High Dew Point,...,poi_health,poi_leisure,poi_catering,poi_accommodation,poi_shopping,poi_money,poi_tourism,poi_miscpoi,poi_traffic,poi_other
2021-01-06,3.0,Sunny,36.59,34,39,31,40,15,32.61,36,...,37,14,107,15,29,15,140,1131,620,19
2021-01-07,2.0,Showers,37.54,34,41,31,40,16,35.33,37,...,37,14,107,15,29,15,140,1131,620,19
2021-01-08,114.0,Showers,35.96,30,41,31,40,7,34.09,36,...,37,14,107,15,29,15,140,1131,620,19
2021-01-09,58.0,Foggy,32.43,25,43,31,40,8,29.57,37,...,37,14,107,15,29,15,140,1131,620,19
2021-01-10,49.0,Cloudy,36.24,28,41,31,40,12,33.07,37,...,37,14,107,15,29,15,140,1131,620,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,92.0,Showers,45.07,43,46,33,42,12,43.57,45,...,37,14,107,15,29,15,140,1131,620,19
2021-12-20,232.0,Cloudy,41.63,32,45,33,42,7,35.17,39,...,37,14,107,15,29,15,140,1131,620,19
2021-12-21,14.0,Sunny,31.80,27,43,33,42,5,28.54,32,...,37,14,107,15,29,15,140,1131,620,19
2022-01-20,5.0,Cloudy,39.59,36,43,31,40,30,32.93,37,...,37,14,107,15,29,15,140,1131,620,19


In [158]:
M_train.columns

Index(['total_litter', 'weather', 'Actual Mean Temperature',
       'Actual Min Temperature', 'Actual Max Temperature',
       'Average Min Temperature', 'Average Max Temperature',
       'Actual Max Wind Speed', 'Actual Dew Point', 'Actual High Dew Point',
       'Actual Low Dew Point', 'Actual Sea Level Pressure', 'poi_public',
       'poi_health', 'poi_leisure', 'poi_catering', 'poi_accommodation',
       'poi_shopping', 'poi_money', 'poi_tourism', 'poi_miscpoi',
       'poi_traffic', 'poi_other'],
      dtype='object')

## 将数值型数据一同归一化

In [159]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [160]:
M_train.loc[:,'Actual Mean Temperature':'poi_other'].values.reshape(-1,1)

array([[  36.59],
       [  34.  ],
       [  39.  ],
       ...,
       [1131.  ],
       [ 620.  ],
       [  19.  ]])

In [161]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(M_train.loc[:,'Actual Mean Temperature':'poi_other'].values.reshape(-1, 1))

In [162]:
mm_wp=mm_data.reshape(228,21)

In [163]:
Counter(mm_wp.flatten())

Counter({0.03235190097259063: 1,
         0.030061892130857647: 37,
         0.034482758620689655: 43,
         0.027409372236958444: 22,
         0.03536693191865606: 19,
         0.01326259946949602: 484,
         0.02883289124668435: 1,
         0.03183023872679045: 48,
         0.028293545534924844: 35,
         0.02659593280282935: 12,
         0.11229000884173297: 228,
         0.03271441202475685: 269,
         0.012378426171529618: 253,
         0.09460654288240494: 228,
         0.02564102564102564: 229,
         0.12378426171529619: 228,
         1.0: 228,
         0.5481874447391688: 228,
         0.016799292661361626: 229,
         0.033191865605658705: 1,
         0.036251105216622455: 46,
         0.014146772767462422: 26,
         0.03123784261715296: 1,
         0.026489832007073385: 12,
         0.031794871794871796: 1,
         0.02652519893899204: 12,
         0.006189213085764809: 5,
         0.030141467727674627: 1,
         0.02664014146772767: 16,
         0.0286

In [164]:
# 合并归一化后的数据
mm_wp_df=pd.DataFrame(mm_wp)
mm_wp_df=mm_wp_df.set_index(wtdf.index)
wt_encoded_df=pd.DataFrame(wt_encoded)
wt_encoded_df=wt_encoded_df.set_index(wtdf.index)
ML=pd.concat([wt_encoded_df,mm_wp_df,M_train['total_litter']],axis=1)

In [165]:
ML

,0,1,2,3,0,1,2,3,4,5,...,12,13,14,15,16,17,18,19,20,total_litter
2021-01-06,0.0,0.0,0.0,1.0,0.032352,0.030062,0.034483,0.027409,0.035367,0.013263,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,3.0
2021-01-07,0.0,0.0,1.0,0.0,0.033192,0.030062,0.036251,0.027409,0.035367,0.014147,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,2.0
2021-01-08,0.0,0.0,1.0,0.0,0.031795,0.026525,0.036251,0.027409,0.035367,0.006189,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,114.0
2021-01-09,0.0,1.0,0.0,0.0,0.028674,0.022104,0.038019,0.027409,0.035367,0.007073,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,58.0
2021-01-10,1.0,0.0,0.0,0.0,0.032042,0.024757,0.036251,0.027409,0.035367,0.010610,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,0.0,0.0,1.0,0.0,0.039850,0.038019,0.040672,0.029178,0.037135,0.010610,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,92.0
2021-12-20,1.0,0.0,0.0,0.0,0.036808,0.028294,0.039788,0.029178,0.037135,0.006189,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,232.0
2021-12-21,0.0,0.0,0.0,1.0,0.028117,0.023873,0.038019,0.029178,0.037135,0.004421,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,14.0
2022-01-20,1.0,0.0,0.0,0.0,0.035004,0.031830,0.038019,0.027409,0.035367,0.026525,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,5.0


In [166]:
ML_array=np.array(ML)

In [167]:
np.save('Hoorn_array2', ML)

## poi提取

In [173]:
poi1=pd.read_csv('./data/poi1/poi1.csv')

In [169]:
poi1['geometry']

0      POINT (5.0817843 52.6456291)
1      POINT (5.0961325 52.6457239)
2      POINT (5.0812344 52.6458317)
3      POINT (5.0968648 52.6460559)
4      POINT (5.0853898 52.6531698)
                   ...             
450    POINT (5.0931458 52.6533975)
451    POINT (5.0932666 52.6531166)
452    POINT (5.0932798 52.6532018)
453    POINT (5.0934684 52.6531761)
454    POINT (5.0936391 52.6532932)
Name: geometry, Length: 455, dtype: object

In [170]:
points['geometry'].astype(str)

0       POINT (5.0624231 52.6350804)
1       POINT (5.0085054 52.6350184)
2         POINT (5.0578571 52.63695)
3       POINT (5.0572411 52.6381662)
4       POINT (5.0523931 52.6417891)
                    ...             
2397     POINT (5.0472634 52.647675)
2398    POINT (5.0469239 52.6475957)
2399    POINT (5.0465883 52.6476558)
2400    POINT (5.0473351 52.6479847)
2401    POINT (5.0471709 52.6476163)
Name: geometry, Length: 2402, dtype: object

In [171]:
poi1_flag=points['geometry'].astype(str).isin(poi1['geometry'])

In [172]:
poi1_flag.value_counts()

False    1947
True      455
Name: geometry, dtype: int64

In [174]:
poi1=points[poi1_flag].reset_index(drop=True)# 第一类poi

In [175]:
poi1.to_file('./data/poi1.shp')# 输出为shapefile文件

In [176]:
poi1[['osm_id','type','geometry']].set_index(['osm_id'])

,type,geometry
osm_id,,
47077200,traffic,POINT (5.08178 52.64563)
47080156,traffic,POINT (5.09613 52.64572)
47081177,traffic,POINT (5.08123 52.64583)
47081536,traffic,POINT (5.09686 52.64606)
47097804,traffic,POINT (5.08539 52.65317)
...,...,...
49020516,miscpoi,POINT (5.09315 52.65340)
49020522,miscpoi,POINT (5.09327 52.65312)
49020523,miscpoi,POINT (5.09328 52.65320)


In [198]:
poi2=pd.read_csv('./outdata/poi2/poi2.csv')

In [200]:
poi2_flag=points['geometry'].astype(str).isin(poi2['geometry'])

In [201]:
poi2_flag.value_counts()

False    1433
True      969
Name: geometry, dtype: int64

In [202]:
poi2=points[poi2_flag].reset_index(drop=True)# 第二类poi

In [203]:
poi2.to_file('./outdata/poi2/poi2.shp')# 输出为shapefile文件

In [204]:
poi3=pd.read_csv('./outdata/poi3/poi3.csv')

In [205]:
poi3_flag=points['geometry'].astype(str).isin(poi3['geometry'])

In [206]:
poi3_flag.value_counts()

False    2360
True       42
Name: geometry, dtype: int64

In [207]:
poi3=points[poi3_flag].reset_index(drop=True)# 第二类poi

In [208]:
poi3.to_file('./outdata/poi3/poi3.shp')# 输出为shapefile文件

## 时间分段标签

In [177]:
is_weekly=df_date.weekday==6 # 一周结束

In [178]:
df_date.weekday==(5 or 6)

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
        True, False,

In [179]:
holiday=pd.to_datetime(['2021-01-01','2021-04-02','2021-04-04','2021-04-27','2021-05-04','2021-05-05','2021-05-13','2021-05-23','2021-05-24',
         '2021-12-05','2021-12-06','2021-12-24','2021-12-25','2021-12-26','2021-12-31',
        '2022-01-01','2022-04-02','2022-04-04','2022-04-27','2022-05-04','2022-05-05','2022-05-26','2022-06-05','2022-06-06',
         '2022-12-05','2022-12-06','2022-12-24','2022-12-25','2022-12-26','2022-12-31'])

In [180]:
is_holiday=df_date.isin(holiday)

In [181]:
is_holiday=np.array(is_holiday)+np.array(df_date.weekday==(5 or 6))

In [182]:
Date=pd.DataFrame(np.zeros([228,3]),columns=['daily','weekly','holiday'])

In [183]:
Date=Date.set_index(df_date)

In [184]:
ML1=pd.concat([Date,ML],axis=1)

In [185]:
ML1

,daily,weekly,holiday,0,1,2,3,0,1,2,...,12,13,14,15,16,17,18,19,20,total_litter
2021-01-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.032352,0.030062,0.034483,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,3.0
2021-01-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.033192,0.030062,0.036251,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,2.0
2021-01-08,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.031795,0.026525,0.036251,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,114.0
2021-01-09,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.028674,0.022104,0.038019,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,58.0
2021-01-10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.032042,0.024757,0.036251,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.039850,0.038019,0.040672,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,92.0
2021-12-20,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.036808,0.028294,0.039788,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,232.0
2021-12-21,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.028117,0.023873,0.038019,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,14.0
2022-01-20,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.035004,0.031830,0.038019,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,5.0


In [186]:
ML1['daily']=1.0

In [187]:
ML1[is_weekly].index

DatetimeIndex(['2021-01-10', '2021-01-31', '2021-02-21', '2021-02-28',
               '2021-03-07', '2021-03-14', '2021-03-21', '2021-04-11',
               '2021-04-18', '2021-04-25', '2021-05-02', '2021-05-09',
               '2021-05-16', '2021-05-30', '2021-06-06', '2021-06-13',
               '2021-06-20', '2021-06-27', '2021-07-04', '2021-07-18',
               '2021-07-25', '2021-08-01', '2021-08-29', '2021-09-05',
               '2021-09-12', '2021-09-26', '2021-10-03', '2021-10-10',
               '2021-10-17', '2021-10-24', '2021-11-14', '2021-11-21',
               '2021-11-28', '2021-12-12', '2021-12-19', '2022-03-20'],
              dtype='datetime64[ns]', freq=None)

In [188]:
ML1.loc[is_weekly,'weekly']=1.0

In [189]:
ML1['weekly'].value_counts()

0.0    192
1.0     36
Name: weekly, dtype: int64

In [190]:
ML1.loc[is_holiday,'holiday']=1.0

In [191]:
ML1['holiday'].value_counts()

0.0    192
1.0     36
Name: holiday, dtype: int64

In [192]:
ML1

,daily,weekly,holiday,0,1,2,3,0,1,2,...,12,13,14,15,16,17,18,19,20,total_litter
2021-01-06,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.032352,0.030062,0.034483,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,3.0
2021-01-07,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.033192,0.030062,0.036251,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,2.0
2021-01-08,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.031795,0.026525,0.036251,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,114.0
2021-01-09,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.028674,0.022104,0.038019,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,58.0
2021-01-10,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.032042,0.024757,0.036251,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-19,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.039850,0.038019,0.040672,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,92.0
2021-12-20,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.036808,0.028294,0.039788,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,232.0
2021-12-21,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.028117,0.023873,0.038019,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,14.0
2022-01-20,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.035004,0.031830,0.038019,...,0.012378,0.094607,0.013263,0.025641,0.013263,0.123784,1.0,0.548187,0.016799,5.0


In [ ]:
ML1.to_csv('./Baseline_data.csv')